# Оценка эмоциональной окраски сообщений

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
%config IPCompleter.greedy = True

## Загрузка данных

In [2]:
data = pd.read_excel('banki_data_sigma.xlsx')

In [3]:
# 5 примеров строк
data.head(5)

,text,label
0,отвратительное обслуживание был у меня вклад в...,0
1,мнение о банке изменилось в худшую сторону это...,0
2,банк поступил красиво у меня дебетовая карта б...,1
3,прошу принять меры по исправлению ситуации бан...,0
4,спокойно и качественно пользуюсь услугами сигм...,1


In [4]:
# пример отзыва
print data.text[0]

отвратительное обслуживание был у меня вклад в сигме. решила я его перевести в другой банк. был открыт экспресс-счет. зная как работают сотрудники колл-центра сигмы и насколько они квалифицированы, я позвонила 4 раза разным менеджерам, уточнить комиссию при переводе. всеми ими была озвучена цифра в 2000 руб. (максимальная при переводе с экспресс счета в другой банк).  будучи в офисе до замоскворечье 02 декабря, консультируясь по вопросу вкладов и переводов с сотрудником ириной м. (к сожалению, не записала фамилию), ею была озвучена также цифра в 2000 руб. придя в банк  07/12/15  для осуществления данного перевода, сотрудником с-ским д.е. была озвучена цифра в 23000 руб.! на 21000 больше! его коллега (маленькая худенькая девчушка, к сожалению, в силу эмоционального состояния не записала ее имя) также начала мне доказывать, что комиссия за данную операцию именно 23 тыс.руб. на аргументы о 4-х сотрудниках колл-центра и о сотруднике этого же отделения, они вдвоем хлопали ресницами и настаи

## Размеры классов

In [5]:
data.groupby("label").count()

,text
label,
0,4921
1,1150


## Наиболее часто встречающиеся слова

In [6]:
# для позитивных отзывов
positive_reviews = ' '.join(data.text[data.label == 1])
positive_counter = Counter(positive_reviews.split())

for word in positive_counter.most_common(10):
    print word[0], word[1]

в 9637
и 8819
не 5484
на 4745
что 3961
я 3719
с 3593
- 2796
по 2427
мне 1960


In [7]:
# для негативных
negative_reviews = ' '.join(data.text[data.label == 0])
negative_counter = Counter(negative_reviews.split())

for word in negative_counter.most_common(10):
    print word[0], word[1]

в 53074
и 43768
не 36077
на 28249
что 27191
я 26467
с 18591
мне 14603
по 14069
- 13632


## Обучаем первую модель

In [8]:
# преобразуем текст в числа
tfidf = TfidfVectorizer()
f = tfidf.fit_transform(data["text"])
print f.shape

(6071, 72051)


In [9]:
pd.DataFrame(f[:5, :].todense(), columns=tfidf.get_feature_names()).head()

,00,000,0000,00000,00034,00079700,0008,000р,001,0015,...,ящичках,ящичке,яя,яяявно,ёе,ёлке,ёлы,ёпрст,ёрничать,ёё
0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.19893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, data["label"], data, test_size=0.2, stratify=data["label"], random_state=42
)
clf = LogisticRegressionCV(penalty='l2', cv=4, n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 7.67 s, sys: 226 ms, total: 7.9 s
Wall time: 8.62 s


In [11]:
# добавляем колонку с вероятностью положительного отзыва
teDF = teDF.copy()
teDF["predicted_proba"] = map(str, clf.predict_proba(tfidf.transform(teDF["text"]))[:, 1])

In [12]:
teDF.tail(10)

,text,label,predicted_proba
1864,здравствуйте! являюсь клиентом банка уже мног...,0,0.00286325566485
2666,проблема при получении справки о закрытии кред...,0,0.097255931984
6014,согласен!!! полностью поддерживаю димитрия!!! ...,0,0.620411770872
4367,вымогательство денег пользовался кредитной кар...,0,0.00129370420101
1326,дают неверные реквизиты счета было необходимо ...,0,0.00081806394855
4483,"услуга сигма-чек здравствуйте. как я посмотрю,...",0,0.00103480256902
3114,навязывание услуг уже несколько лет являюсь кл...,0,0.0136346614423
3346,даже железобетонному терпению когда-то приходи...,0,0.00208515742497
5892,очень доволен банком в мае этого года взял тех...,1,0.999865741146
1673,"не предупрежден - не вооружен, даже нагло обма...",0,0.000225690422633


## Потестируем модель

In [13]:
clf.predict_proba(tfidf.transform([u"как всегда плохо спасибо"]))[:,1]

array([ 0.99999848])

In [14]:
clf.predict_proba(tfidf.transform([u"ужасно"]))[:,1]

array([ 0.06402277])

In [15]:
clf.predict_proba(tfidf.transform([u"плохо"]))[:,1]

array([ 0.11384584])

In [16]:
clf.predict_proba(tfidf.transform([u"помогли"]))[:,1]

array([ 0.76318487])

## Формальные метрики качества

In [17]:
print "train accuracy:", accuracy_score(trY, clf.predict(trX))
print "test accuracy:", accuracy_score(teY, clf.predict(teX))

train accuracy: 1.0
test accuracy: 0.921810699588


In [18]:
print classification_report(teY, clf.predict_proba(teX)[:,1] > 0.1)

             precision    recall  f1-score   support

          0       0.96      0.93      0.94       985
          1       0.73      0.84      0.78       230

avg / total       0.92      0.91      0.91      1215



## Посмотрим на веса модели

In [19]:
coefs = sorted(zip(list(np.array(tfidf.get_feature_names())[clf.coef_[0] != 0]), clf.coef_[0][clf.coef_[0] != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])

In [20]:
coefs.head(30)

,ngram,weight
0,спасибо,26.388012
1,благодарность,15.489216
2,быстро,14.156494
3,приятно,13.197226
4,очень,11.756937
5,проблем,10.847594
6,доволен,9.726652
7,оперативно,9.689212
8,нравится,9.520648
9,понравилось,8.750875


In [21]:
coefs.tail(30)[::-1]

,ngram,weight
64238,не,-8.347115
64237,клиентов,-7.508055
64236,ответ,-7.236294
64235,вы,-7.017829
64234,говорят,-6.857047
64233,звонки,-6.399363
64232,потом,-5.777867
64231,нет,-5.644717
64230,никто,-5.631515
64229,евро,-5.271084


## Как улучшить качество модели?
* Фильтрация стоп-слов
* Приведение слов к нормальной форме
* Генерация дополнительных признаков: статистики на основе пар слов, n-gramm, длина отзыва и пр.
* Рассмотрение других моделей
* Настройка параметров моделей

## Попробуем фильтровать стоп-слова

In [22]:
# посчитаем встречаемость всех слов
all_words = ' '.join(data.text)
word_counts = Counter(all_words.split())

In [23]:
# самые встречаемые
for word in word_counts.most_common(20):
    print word[0], word[1]

в 62711
и 52587
не 41561
на 32994
что 31152
я 30186
с 22184
мне 16563
по 16496
- 16428
а 10098
за 9621
меня 9301
как 9224
но 9141
это 8794
у 8177
все 7756
о 7581
банка 7026


In [24]:
# наименее встречаемые
for word in sorted(word_counts.items(), key=lambda x: x[1])[:20]:
    print word[0], word[1]

(потратила 1
(зп-клиент). 1
(тяжело, 1
"сигма-хранителем"? 1
а-мобайл 1
функционировал 1
«массовый 1
вынужденной 1
ростов-на-дону). 1
предусматривающие 1
сухого: 1
подозрения): 1
возвращена). 1
подчеркнула 1
сделать!), 1
трансфер 1
ленточка: 1
(http://www.banki.ru/services/questions-answers/?questionid=7582990) 1
уголовниками. 1
городских, 1


In [25]:
good_words = [word for word, count in word_counts.items() if 10 <= count <= 7000]

In [26]:
tfidf = TfidfVectorizer(vocabulary=good_words)
f = tfidf.fit_transform(data["text"])
print f.shape

(6071, 14130)


In [27]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, data["label"], data, test_size=0.2, stratify=data["label"], random_state=42
)
clf = LogisticRegressionCV(penalty='l2', cv=4, n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 5.94 s, sys: 163 ms, total: 6.11 s
Wall time: 7.23 s


In [28]:
print "train accuracy:", accuracy_score(trY, clf.predict(trX))
print "test accuracy:", accuracy_score(teY, clf.predict(teX))

train accuracy: 1.0
test accuracy: 0.925925925926


## Добавим словосочетания

In [29]:
tfidf = TfidfVectorizer(min_df=10, max_df=7000, ngram_range=(1, 2))
f = tfidf.fit_transform(data["text"])
print f.shape

(6071, 27051)


In [30]:
%%time
# делим на обучение и тест
trX, teX, trY, teY, trDF, teDF = train_test_split(
    f, data["label"], data, test_size=0.2, stratify=data["label"], random_state=42
)
clf = LogisticRegressionCV(penalty='l2', cv=4, n_jobs=1, solver='liblinear')
clf.fit(trX, trY)

CPU times: user 8.19 s, sys: 236 ms, total: 8.43 s
Wall time: 9.03 s


In [31]:
print "train accuracy:", accuracy_score(trY, clf.predict(trX))
print "test accuracy:", accuracy_score(teY, clf.predict(teX))

train accuracy: 1.0
test accuracy: 0.930864197531


In [32]:
coefs = sorted(zip(list(np.array(tfidf.get_feature_names())[clf.coef_[0] != 0]), clf.coef_[0][clf.coef_[0] != 0]),
               key=lambda x: -x[1])
coefs = pd.DataFrame(coefs, columns=["ngram", "weight"])

In [33]:
coefs.head(30)

,ngram,weight
0,спасибо,32.621374
1,быстро,19.206290
2,благодарность,19.102888
3,приятно,16.784856
4,очень,15.252887
5,проблем,13.435192
6,доволен,12.745233
7,оперативно,12.062838
8,нравится,11.757259
9,отзыв,10.895727


In [34]:
coefs.tail(30)[::-1]

,ngram,weight
27050,не,-9.443269
27049,клиентов,-8.781736
27048,вы,-8.664855
27047,не работает,-8.389955
27046,говорят,-8.386884
27045,ответ,-8.065434
27044,потом,-7.781764
27043,звонки,-7.752011
27042,евро,-7.224810
27041,опять,-6.965748
